In [1]:
import geopandas as gpd
from osgeo import ogr
import pandas as pd
import tifffile as tiff #needed for the tif data for perry county
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix
from math import sin, cos, sqrt, atan2, radians
import sys
from shapely.geometry import Polygon, box, Point, LineString, MultiLineString
import pickle
import gurobipy as gp
from gurobipy import GRB
import tifffile as tiff #needed for the tif data for perry county
import xlwt
from xlwt import Workbook
from shapely.ops import snap, split, nearest_points
#from shapely.geometry import MultiPoint, LineString
#from dbfread import DBF
import osmnx as ox
import networkx as nx
import math
import gurobipy as gp
from gurobipy import GRB
import os
import csv
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt

In [2]:
# calculate distance
def haversinedist(lat1, lon1, lat2, lon2):
    R = 6373.0
    
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c
    return distance #unit in km

In [3]:
# read data
def readClusterFile(fileID):
    file = np.genfromtxt(fileID, delimiter=",", skip_header = 1)
    file = file[:,1:]
    return file
clusterfile = 'Centralized_elevcluster' + str(1) + '.csv'

In [4]:
os.chdir("/Users/yuelanzhu/Downloads/Research/my code file")
# read data
def readClusterFile(fileID):
    file = np.genfromtxt(fileID, delimiter=",", skip_header = 1)
    file = file[:,1:]
    return file
clusterfile = 'Centralized_elevcluster' + str(1) + '.csv'
building_coords = readClusterFile(clusterfile)

In [5]:
# pipe parameters
pipesize = [0.05, 0.06, 0.08, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.6] #m
pipesize_str, pipecost = gp.multidict({'0.05': 8.7, '0.06': 9.5, '0.08': 11, \
                                                   '0.1': 12.6, '0.15': 43.5,'0.2': 141, '0.25': 151, '0.3': 161, '0.35':230, '0.4': 246, '0.45':262, 
                                                   '0.5':292, '0.6':315}) #$/m
excavation = 25
bedding_cost_sq_ft = 6
capital_cost_pump_station = 166000
ps_flow_cost = 0.19 * (1/4.381E-8) #$/gpd to $/m3s-1
ps_OM_cost = 175950

In [6]:
# creat MST

graph = []
mstree = []
def createMSTnx(dataframe,n):
    #only takes longitude and latitude from the cluster data, note we also have elevation and stuff there too
    #coordinates = twoDcluster
    cluster = AgglomerativeClustering(n_clusters=n, affinity='euclidean', linkage='ward')
    cluster.fit_predict(building_coords[:,1:2])
    # add cluster to dataframe
    dataframe['cluster.labels_'] = cluster.labels_.tolist()
    for k in range(n):
        cluster_label = dataframe[dataframe['cluster.labels_'] == k]
        cluster_label = cluster_label[['longitude','latitude']]
        latlon = cluster_label.to_numpy()
        nrows, ncols = latlon.shape
        #creates graph
        graph.append(nx.Graph())
        weights = []
        #distance in km as weight between each point in the graph
        for i in range(nrows):
            graph[k].add_node(i,pos=(latlon[i,0],latlon[i,1]))
            for j in range(i+1,nrows):
                dist = haversinedist(latlon[i,1], latlon[i,0], latlon[j,1], latlon[j,0])
                weights.append(dist)
                graph[k].add_edge(i,j, weight = dist)
        #creates MST
        mstree.append(nx.minimum_spanning_tree(graph[k]))
    return mstree, graph

In [7]:
#convert data to dataframe
df = pd.DataFrame(building_coords,
                  columns = ['longitude','latitude','elevation'])
#createMSTnx(df,10)

In [8]:
def num_c_2(df, num_c, cluster_labels, pipesize, pipecost, water_demand, HLmax, excavation, bedding_cost_sq_ft, capital_cost_pump_station, ps_OM_cost):

    # select highest and lowest nodes in cluster
    tank = df.loc[df.groupby(['cluster.labels_'])['elevation'].idxmax()].reset_index(drop=True)
    trem = df.loc[df.groupby(['cluster.labels_'])['elevation'].idxmin()].reset_index(drop=True)
                      
     # find label of tank node in cluster
    tank_lon = str(tank[tank['cluster.labels_'] == cluster_labels][['longitude']].iat[0,0])
    tank_lat = str(tank[tank['cluster.labels_'] == cluster_labels][['latitude']].iat[0,0])
    global tank_node
    for i in mstree[cluster_labels].nodes():
        if str(mstree[cluster_labels].nodes[i]['pos'][0]) == tank_lon and str(mstree[cluster_labels].nodes[i]['pos'][1]) == tank_lat:
            tank_node = i                  
    #create flow direction dict
    links = list(mstree[cluster_labels].edges)
    flow_dir = {}
    for i,j in links:
        if len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=i)) < len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=j)):
        # if node i is nearer to tank
           flow_dir[i,j]=(i,j)
        if len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=i)) > len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=j)):
         # if node j is nearer to tank
           flow_dir[i,j]=(j,i) 
    # find nodes for inflow and outflow
    innode = {}
    outnode = {}
    for i,j in links:
        if len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=i)) > len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=j)):
           # if node j is nearer to tank
            innode.setdefault(i, [])
            innode[i].append(j)
            outnode.setdefault(j, [])
            outnode[j].append(i)
        else:
           # if node i is nearer to tank
            innode.setdefault(j, [])
            innode[j].append(i)
            outnode.setdefault(i, [])
            outnode[i].append(j)
    for i in mstree[cluster_labels].nodes():
        innode.setdefault(i,[])
        outnode.setdefault(i, [])
   # directed graph
    mstree_dir = nx.DiGraph()
    mstree_dir.add_nodes_from(mstree[cluster_labels])
    mstree_dir.add_edges_from(flow_dir.values())
    weights = {}
    for i in list(mstree_dir.edges()):
        weights[i] = None
    for i,j in flow_dir:
        if flow_dir[i,j]==(i,j):
            weights[i,j]=mstree[cluster_labels].edges[i,j]['weight']
        if flow_dir[i,j]==(j,i):
            weights[j,i]=mstree[cluster_labels].edges[i,j]['weight']
    nx.set_edge_attributes(mstree_dir, values = 1, name = 'weight')
    nx.set_edge_attributes(mstree_dir, values = weights, name = 'weight')
    # determine iterate order
    walk = {}
    for i,j in mstree_dir.edges():
        walk[i,j] = len(nx.shortest_path(mstree_dir ,source=tank_node, target=j))
    sorted_walk = dict(sorted(walk.items(), key=lambda item: item[1], reverse = True))
    # pipe flow
    pipe_flow = {}
    for i,j in sorted_walk:
        if len(outnode[j]) == 0:
            pipe_flow[i,j] = 1.095E-5
        else:
            pipe_flow[i,j] = sum(pipe_flow[m,n] for m,n in sorted_walk if m == j) + 1.095E-5 #250 gpd to m3/s
    # head loss at largest dia        
    head_loss = {}
    for i in list(mstree_dir.edges()):
        head_loss[i] = None
    for i,j in list(mstree_dir.edges()):
        head_loss[i,j] = 10700*(pipe_flow[i,j]/140)**1.852*0.6*1000*mstree_dir[i][j]["weight"]
    # flow, distance and head loss for each edge
    edge_list = {}
    for i,j in links:
        if (i,j) == flow_dir[i,j]:
            edge_list[i,j] =[pipe_flow[i,j],mstree[cluster_labels][i][j]["weight"], head_loss[i,j]]
        else:
            edge_list[j,i] =[pipe_flow[j,i],mstree[cluster_labels][i][j]["weight"], head_loss[j,i]]
   #node elevation excavation in meters
   #upper bound is arbritrary maximum depth assuming 1 foot or 0.3048 meters of cover beneath the surface is needed for the pipes
   #a lower bound variable is created but not used. In future models might need to implement that depending on the site (digging too deep for excavation is not feasible for many projects)
    cluster = df[df['cluster.labels_'] == cluster_labels]
    cluster_elv = cluster[['elevation']]
    elevation_ub = dict()
    elevation_lb = dict()
    for i in range(len(cluster_elv)):
        GE=float(str(cluster_elv.values.tolist()[i])[1:-1])
        elevation_ub[i] = GE - 0.3
        elevation_lb[i] = GE - 10 #10
    GE=[]
    for i in range(len(cluster_elv)):
        GE.append(float(str(cluster_elv.values.tolist()[i])[1:-1]))
    #for i in range(mstree[cluster_labels].number_of_nodes()):
        #elevation_ub[i] = cluster_elv.iloc[[0]] - 0.3048
        #elevation_lb[i] = cluster_elv.iloc[[0]] - 30
        
    m = gp.Model('pipe and pump distribution')
    m.Params.timeLimit = 12000
   # binary variable indicating if at link ij pipe of diameter k is implemented
    d = m.addVars(mstree_dir.edges(), pipesize, vtype = GRB.BINARY, name = "diameter")
   #continuous variable representing the hydro loss in link ij.
    HL = m.addVars(mstree_dir.edges(),lb = 0, vtype = GRB.CONTINUOUS, name = "hydro loss")
    # hydro gradient
    #J = m.addVars(mstree[cluster_labels].edges(),lb = 0, vtype = GRB.CONTINUOUS, name = "hydro gradient")
   # binary variable indicating if at link ij the pump of type m is implemented.
    p = m.addVars(mstree_dir.edges(), vtype = GRB.BINARY, name = "PUMP")
   #continuous variable representing the pressure injected by the pump at link ij.
    pr = m.addVars(mstree_dir.edges(),lb = 0, vtype = GRB.CONTINUOUS, name = "PRESSURE BY PUMP")
   #pipe elevations at node i 
    e = m.addVars(mstree_dir.nodes(), lb = 0, vtype = GRB.CONTINUOUS, name = 'In Node Elevation')
   #continuous variable representing the head pressure at node i
    H = m.addVars(mstree_dir.nodes(),lb = 0, vtype = GRB.CONTINUOUS, name = "HEAD PRESSURE")
    for i,j in mstree_dir.edges():
#Hazen Williams equation, A = 10700, C=140, P472
        #m.addConstr(J[i,j] == 
                #10700*(edge_list[i,j][0]/140)**1.852*gp.quicksum(d[i,j,k]/(k**4.8704) for k in pipesize)
               #, "Hydro loss")
        m.addConstr(HL[i,j] == 10700*(edge_list[i,j][0]/140)**1.852*gp.quicksum(d[i,j,k]/(k**4.8704) for k in pipesize)*1000*mstree_dir[i][j]["weight"])
        m.addConstr(HL[i,j] <= HLmax)
#only one pipe per edge
        m.addConstr((d.sum(i, j, '*') == 1.0), "single size chosen")
#only one pump per edge
        m.addConstr((p.sum(i, j, '*') <= 1.0), "single type chosen")
    
#injected pressure pr[i,j] must be less than a huge number.
        #m.addConstr(pr[i,j]<=p[i,j]*10**6)
    
    # injected pressure pr[i,j] must be less than the capacity of pump.
        m.addConstr(pr[i,j] <= p[i,j]*100, "pressure by pump less than 100 m ")
    # energy requirement
        m.addConstr(H[j] == H[i] - HL[i,j] + e[i] - e[j] + pr[i,j], "Energy balance")
    
        #if len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=i)) < len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=j)):
        # if node i is nearer to tank        
            #m.addConstr(H[j] == H[i] -HL[i,j] + e[i] - e[j] + pr[i,j], "Energy balance")
        #if len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=i)) > len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=j)):
        # if node j is nearer to tank        
            #m.addConstr(H[i] == H[j] -HL[i,j] + e[j] - e[i] + pr[i,j], "Energy balance")
# Velocity must be between 0.6 and 3 m/s to 30
        m.addConstr((
           edge_list[i,j][0] <= ((3.14/4)*gp.quicksum(d[i,j,k]*k**2 for k in pipesize)) * 3), "Velocity Max Constr"+ str([i,j]))
        #m.addConstr((
           #edge_list[i,j][0] >= ((3.14/4)*gp.quicksum(d[i,j,k]*k**2 for k in pipesize)) * 0.6), "Velocity Min Constr" + str([i,j]))  
    for i in range(len(cluster_elv)):
#pipe elevation must be betwenn lb and ub
        m.addConstr(e[i] >= elevation_lb[i], "PIPE ELV LB")
        m.addConstr(e[i] <= elevation_ub[i], "PIPE ELV UB")
    
        m.addConstr(H[i] >= 31.646232670592248, "Min head pressure") #45 psi, covert it to m
        m.addConstr(H[i] <= 56.26, "Max head pressure") #80 psi, covert it to m   
        
        
    #m.addConstr(H[tank_node] == 31.646232670592248)
                      
                      
    # pipe cost 
    obj1 = gp.quicksum(1000 * mstree_dir[i][j]["weight"] * gp.quicksum(pipecost[str(k)] * d[i, j, k] 
                                                           for k in pipesize) for i,j in mstree_dir.edges())
    
    # Capital cost of pump stations
    obj2 = gp.quicksum(p.sum(i, j) * (capital_cost_pump_station + ps_flow_cost*edge_list[i,j][0]) for i,j in mstree_dir.edges())
    #Operation and maintenance costs
    obj3 = gp.quicksum(p.sum(i, j)*ps_OM_cost for i, j in mstree_dir.edges())
    # Excavation cost
    obj4 = gp.quicksum((1 + gp.quicksum(d[i,j,k]*k for k in pipesize)) *1000* mstree_dir[i][j]["weight"] * bedding_cost_sq_ft + excavation * \
                               (1 + gp.quicksum(d[i,j,k]*k for k in pipesize)) *1000* mstree_dir[i][j]["weight"] * 0.5 * ((GE[i] - e[i]) + (GE[j] - e[j])) for i, j in mstree_dir.edges())

    # Capital cost of pump stations
    #obj2 = gp.quicksum(1000 * mstree_dir[i][j]["weight"] * gp.quicksum(pumpcost[pumptype.index(m)] * p[i, j, m] 
                                                           #for m in pumptype) for i,j in mstree_dir.edges()) #+penalty

    #Operation and maintenance costs
    #obj3 = gp.quicksum(p.sum(i, j)*ps_OM_cost for i, j in mstree_dir.edges())

    obj = obj1+obj2+obj3#+obj4                  
    m.setObjective(obj, GRB.MINIMIZE)
    m.optimize()
    
    # record dia
    #count = 0
    #for i, j in mstree_dir.edges():
        #for k in range(len(pipesize)):
            #if m.getVars()[k+count].X == 1:
                #edge_list[i,j].append(pipesize[k])
        #count = count + len(pipesize)
    #d = []
    #for i,j in mstree_dir.edges():
        #d.append(edge_list[i,j][3])
    #edge_dia = {}
    #count = 0
    #for i,j in links:
        #edge_dia[i,j]=d[count]
        #count = count+1    
    
    #plot
    #pos = nx.spring_layout(mstree[cluster_labels])

    #nx.draw_networkx(mstree[cluster_labels], pos)

    #nx.draw_networkx_labels(mstree[cluster_labels], pos, labels=edge_dia,label_pos=0.5, font_size=8)

    #plt.show()
    
    #save the result
    current_path = os.getcwd()
    folder = current_path +'/num_c'+ str(num_c)
    if not os.path.exists(folder):
        os.makedirs(folder)
    os.chdir(folder)
    
    modelname = str(cluster_labels) + ".csv" 
        #m.write(modelname)
    modelfile = open(modelname, "w")
    #modelfile.write("pipecost" + str(pipecost))
    #modelfile.write('range: %g \n' % bound)
    modelfile.write('tank: %g \n' % tank_node)
    modelfile.write('number of nodes: %g \n' % len(mstree[cluster_labels].nodes()))
    modelfile.write('number of edges: %g \n' % len(mstree[cluster_labels].edges()))
    #modelfile.write('number of continuous variables: %g \n' % (2*len(mstree[cluster_labels].edges()) + 2*len(mstree[cluster_labels].nodes()))
    #modelfile.write('number of binary variables: %g \n' % 2*len(mstree[cluster_labels].edges()))
    modelfile.write('capital_cost_pump_station: %g \n' % capital_cost_pump_station)
    modelfile.write('ps_OM_cost: %g \n' % ps_OM_cost)
    modelfile.write('water_demand: %g \n' % water_demand)
    modelfile.write('Solution Value: %g \n' % m.objVal)
    modelfile.write('Solution Value 1: %g \n' % obj1.getValue())
    modelfile.write('Solution Value 2: %g \n'% obj2.getValue())
    modelfile.write('Solution Value 3: %g \n'% obj3.getValue())
    modelfile.write('Solution Value 4: %g \n'% obj4.getValue())
    #modelfile.write('Solution Value 3: %g \n'% obj3.getValue())                  
    for v in m.getVars():
        modelfile.write('%s %g \n' % (str(v.varName), v.x))
    modelfile.close()
    
    diameter = { k : v.X for k,v in d.items() if v.X ==1}
    diameter = list(diameter.keys())
    v_check = {}
    
    #for i,j in links:
        #if (i,j) == flow_dir[i,j]:
            #for k in range(len(links)):
                #if diameter[k][0] == i and diameter[k][1] == j:
                    #v_check[i,j] = [diameter[k][2],edge_list[i,j][0], edge_list[i,j][0]/(3.14*(diameter[k][2]/2)**2)]
        #else:
            #for k in range(len(links)):
                #if diameter[k][0] == j and diameter[k][1] == i:
                    #v_check[j,i] = [diameter[k][2],edge_list[j,i][0], edge_list[j,i][0]/(3.14*(diameter[k][2]/2)**2)]
    
    return obj.getValue(), obj1.getValue(), obj2.getValue(), obj3.getValue(), obj4.getValue()
                      

In [9]:
#convert data to dataframe
df = pd.DataFrame(building_coords,
                  columns = ['longitude','latitude','elevation'])
g_mst = createMSTnx(df,10)
os.chdir("/Users/yuelanzhu/Downloads/Research/my code file")
num_c_2(df, 10, 9, pipesize, pipecost, 1.095E-5, 1e+5, excavation, bedding_cost_sq_ft, capital_cost_pump_station, ps_OM_cost)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-14
Set parameter TimeLimit to value 12000
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 774 rows, 1262 columns and 3784 nonzeros
Model fingerprint: 0x5a648366
Variable types: 282 continuous, 980 integer (980 binary)
Coefficient statistics:
  Matrix range     [2e-07, 1e+02]
  Objective range  [2e+02, 3e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-05, 1e+05]
Presolve removed 597 rows and 317 columns
Presolve time: 0.00s
Presolved: 177 rows, 945 columns, 1830 nonzeros
Variable types: 119 continuous, 826 integer (826 binary)
Found heuristic solution: objective 1.681917e+07

Root relaxation: objective 3.521920e+04, 177 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Inc

(35395.941387080275, 35395.941387080275, 0.0, 0.0, 774946.5709539168)